In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from rank_bm25 import BM25Okapi

# Récupération du dataset préparé

In [ ]:
######################
# Read recipe inputs #
######################
new_etude_dataset_cleaned_prepared = dataiku.Dataset("new_etude_dataset_cleaned_prepared")
new_etude_dataset_cleaned_prepared_df = new_etude_dataset_cleaned_prepared.get_dataframe()
etudes_dataset_cleaned_prepared = dataiku.Dataset("etudes_dataset_cleaned_prepared")
etudes_dataset_cleaned_prepared_df = etudes_dataset_cleaned_prepared.get_dataframe()

# Vectorisation et pré-traitements finaux

Les prétraitements disponibles à cette étape :
- choix de la taille des ngram
- filtrage des termes apparaissant plus ou moins qu'un nombre ou une propotion donnée
- 

In [0]:
##########################################
# Vectorisation et dernier prétraitement #
##########################################
"""
vect = CountVectorizer(ngram_range=(1,1), max_df=1.0, min_df=1).fit(etudes_dataset_cleaned_prepared_df["docs"])#matrice tf
#vect = TfidfVectorizer(ngram_range=(1,1), max_df=1.0, min_df=1).fit(etudes_dataset_cleaned_prepared_df["docs"])#matrice tf-idf
tf_matrix = vect.transform(etudes_dataset_cleaned_prepared_df["docs"])

preparedQuery = new_etude_dataset_cleaned_prepared_df['doc'][0]
vec_query = vect.transform([preparedQuery])

########################
# Calcul de similarité #
########################
res = cosine_distances(vec_query, tf_matrix)[0]
"""

In [0]:
############################
# Calcul de similarité BM25#
############################
data_tokens = []
for index, line in etudes_dataset_cleaned_prepared_df.iterrows():
    data_tokens.append(line['docs'].split(" "))

bm25 = BM25Okapi(data_tokens)

preparedQuery = new_etude_dataset_cleaned_prepared_df['doc'][0].split(" ")

res = bm25.get_scores(preparedQuery)

In [0]:
similarite_etudes_df = pd.DataFrame({'id':etudes_dataset_cleaned_prepared_df['id'], 'score':res})
similarite_etudes_df.sort_values(by=['score'], inplace=True)

########################
# Write recipe outputs #
########################
similarite_etudes = dataiku.Dataset("similarite_etudes")
similarite_etudes.write_with_schema(similarite_etudes_df)